### Part 1

In [ ]:
WEAPONS = []
ARMORS = []
RINGS = []

class Item:

    __slots__ = ['name', 'cost', 'damage', 'armor']

    def __init__(self, name, cost, damage, armor):
        self.name = name
        self.cost = cost
        self.damage = damage
        self.armor = armor

    def __str__(self):
        return f'Item(name={self.name=}, cost={self.cost}, damage{self.damage}, armor={self.armor})'

    def __repr__(self):
        return str(self)


def add(collection, name, cost, damage, armor):
    collection.append(Item(name, cost, damage, armor))

add(WEAPONS, 'Dagger', 8, 4, 0)
add(WEAPONS, 'Shortsword', 10, 5, 0)
add(WEAPONS, 'Warhammer', 25, 6, 0)
add(WEAPONS, 'Longsword', 40, 7, 0)
add(WEAPONS, 'Greataxe', 74, 8, 0)

add(ARMORS, 'Leather', 13, 0, 1)
add(ARMORS, 'Chainmail', 31, 0, 2)
add(ARMORS, 'Splintmail', 53, 0, 3)
add(ARMORS, 'Bandedmail', 75, 0, 4)
add(ARMORS, 'Platemail', 102, 0, 5)

add(RINGS, 'Damage +1', 25, 1, 0)
add(RINGS, 'Damage +2', 50, 2, 0)
add(RINGS, 'Damage +3', 100, 3, 0)
add(RINGS, 'Defense +1', 20, 0, 1)
add(RINGS, 'Defense +2', 40, 0, 2)
add(RINGS, 'Defense +3', 80, 0, 3)

In [ ]:
from typing import Optional


def validate(weapon: Item, armor: Optional[Item], rings: list[Item]) -> None:
    assert (weapon is not None), f'weapon is None'
    assert (type(weapon) == Item), f'{type(weapon) = }, expected Item'
    assert (armor is None or type(armor) == Item), f'{armor = }, expected None or Item'
    assert (type(rings) == list), f'{type(rings) = }, expected list'
    assert (len(rings) in (0, 1, 2)), f'{len(rings) = }, expected 0, 1, or 2'
    for i, r in enumerate(rings):
        assert (type(r) == Item), f'type(rings[{i}]) = {type(r)}, expected Item'

VALID_CHOICES = []

# one weapon
for weapon in WEAPONS:
    # one armor or none
    for armor in ARMORS + [None]:
        # no rings
        validate(weapon, armor, [])
        VALID_CHOICES.append((weapon, armor, []))
        # one ring
        for ring in RINGS:
            validate(weapon, armor, [ring])
            VALID_CHOICES.append((weapon, armor, [ring]))
        # two rings
        for i in range(len(RINGS)-1):
            for j in range(i+1, len(RINGS)):
                validate(weapon, armor, [RINGS[i], RINGS[j]])
                VALID_CHOICES.append((weapon, armor, [RINGS[i], RINGS[j]]))

print('There are', len(VALID_CHOICES), 'valid choices to evaluate.')

In [ ]:
class Status:

    __slots__ = ['hp', 'dmg', 'ac']

    def __init__(self, hp: int, dmg: int, ac: int):
        self.hp = hp
        self.dmg = dmg
        self.ac = ac


BOSS_HP = 109
BOSS_DMG = 8
BOSS_AC = 2

PLAYER_HP = 100


def init_boss() -> Status:
    return Status(BOSS_HP, BOSS_DMG, BOSS_AC)


def init_player(weapon: Item, armor: Optional[Item], rings: list[Item]) -> Status:
    rv = Status(PLAYER_HP, 0, 0)
    items = [weapon]
    if armor:
        items.append(armor)
    items.extend(rings)
    for item in items:
        rv.ac += item.armor
        rv.dmg += item.damage
    return rv


def calc_total_cost(weapon: Item, armor: Optional[Item], rings: list[Item]) -> int:
    rv = 0
    items = [weapon]
    if armor:
        items.append(armor)
    items.extend(rings)
    for item in items:
        rv += item.cost
    return rv


def calc_dmg(attacker: Status, defender: Status) -> int:
    rv = attacker.dmg - defender.ac
    if rv < 1:
        rv = 1
    return rv


def eval(weapon: Item, armor: Optional[Item], rings: list[Item]) -> tuple[bool, int]:
    boss = init_boss()
    player = init_player(weapon, armor, rings)
    attacker, defender = player, boss
    while boss.hp > 0 and player.hp > 0:
        dmg = calc_dmg(attacker, defender)
        defender.hp -= dmg
        attacker, defender = defender, attacker
    if boss.hp <= 0:
        return True, calc_total_cost(weapon, armor, rings)
    return False, calc_total_cost(weapon, armor, rings)

### Part 1

In [ ]:
from tqdm.notebook import tqdm

lowest_winning_cost = None
associated_setup = None

win_count = 0
lose_count = 0

for weapon, armor, rings in tqdm(VALID_CHOICES):
    player_wins, total_cost = eval(weapon, armor, rings)
    if player_wins:
        win_count += 1
    else:
        lose_count += 1
    if player_wins and (lowest_winning_cost is None or total_cost < lowest_winning_cost):
        lowest_winning_cost = total_cost
        associated_setup = weapon, armor, rings

if lowest_winning_cost is None:
    print(f'ERROR! Did not find any winning setup!')
else:
    print(f'Lowest Winning Cost: {lowest_winning_cost} gold')

### Part 2

In [ ]:
from tqdm.notebook import tqdm

highest_losing_cost = None
associated_setup = None

win_count = 0
lose_count = 0

for weapon, armor, rings in tqdm(VALID_CHOICES):
    player_wins, total_cost = eval(weapon, armor, rings)
    if player_wins:
        win_count += 1
    else:
        lose_count += 1
    if (not player_wins) and (highest_losing_cost is None or total_cost > highest_losing_cost):
        highest_losing_cost = total_cost
        associated_setup = weapon, armor, rings

if highest_losing_cost is None:
    print(f'ERROR! Did not find any losing setup!')
else:
    print(f'Highest Losing Cost: {highest_losing_cost} gold')